In [1]:

import tensorflow as tf
import numpy as np
import gnn.gnn_utils as utils
from gnn.GNN import GNN as GraphNetwork


# import tensorflow as tf
# import numpy as np
# import utils
# import GNNs as GNN
# import Net_Karate as n
# from scipy.sparse import coo_matrix

##### GPU & stuff config
import os

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)


############# training set ################


E, N, labels,  mask_train, mask_test = utils.load_karate()
inp, arcnode, graphnode = utils.from_EN_to_GNN(E, N)



EPSILON = 0.00000001

@tf.function()
def loss(target,output,mask):
    target = tf.cast(target,tf.float32)
    output = tf.maximum(output, EPSILON, name="Avoiding_explosions")  # to avoid explosions
    xent = -tf.reduce_sum(target * tf.math.log(output), 1)

    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    xent *= mask
    lo = tf.reduce_mean(xent)
    return lo

@tf.function()
def metric(output, target,mask):
    correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(target, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask

    return tf.reduce_mean(accuracy_all)




# set input and output dim, the maximum number of iterations, the number of epochs and the optimizer
threshold = 0.001
learning_rate = 0.0001
state_dim = 2
input_dim = inp.shape[1]
output_dim = labels.shape[1]
max_it = 50
num_epoch = 1000

def create_model():

    comp_inp = tf.keras.Input(shape=(input_dim), name="input")
    
    layer = GraphNetwork(input_dim, state_dim, output_dim,                             
                         hidden_state_dim = 5, hidden_output_dim = 5,
                         ArcNode=arcnode,NodeGraph=None,threshold=threshold)
    
    output = layer(comp_inp)
    
    model = tf.keras.Model(comp_inp, output)

    return model,layer


tf.keras.backend.clear_session()
model,GNNLayer = create_model()

# initialize GNN
param = "st_d" + str(state_dim) + "_th" + str(threshold) + "_lr" + str(learning_rate)
print(param)

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
for count in range(num_epoch):

    with tf.GradientTape() as tape:
        
        out = model(inp,training=True)

        loss_value = loss(labels,out, mask=mask_train)

        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        if count % 10 == 0:
            out_val = GNNLayer.predict_node(inp, arcnode)
            loss_value_val = loss(labels,out, mask=mask_test)
            
            print("Epoch ", count)
            print("Training: ", loss_value.numpy())
            print("Test: ", loss_value_val.numpy())

Loading karate club dataset...
st_d2_th0.001_lr0.0001
Epoch  0
Training:  1.4895287
Test:  1.4496232
Epoch  10
Training:  1.4709486
Test:  1.4484726
Epoch  20
Training:  1.451918
Test:  1.4479373
Epoch  30
Training:  1.4323565
Test:  1.4481275
Epoch  40
Training:  1.4122018
Test:  1.4492273
Epoch  50
Training:  1.3914547
Test:  1.4514649
Epoch  60
Training:  1.3702981
Test:  1.4550579
Epoch  70
Training:  1.3493625
Test:  1.4600443
Epoch  80
Training:  1.3300219
Test:  1.4660459
Epoch  90
Training:  1.3140364
Test:  1.4719964
Epoch  100
Training:  1.3020201
Test:  1.4761168
Epoch  110
Training:  1.2924347
Test:  1.4768645
Epoch  120
Training:  1.2834069
Test:  1.4743016
Epoch  130
Training:  1.2744728
Test:  1.4699994
Epoch  140
Training:  1.2657646
Test:  1.4655848
Epoch  150
Training:  1.2572708
Test:  1.461812
Epoch  160
Training:  1.2489384
Test:  1.4586589
Epoch  170
Training:  1.2407547
Test:  1.455803
Epoch  180
Training:  1.2327147
Test:  1.4529902
Epoch  190
Training:  1.22480

In [ ]:
np.sum(labels,axis=0)

In [ ]:
arcnode